# Day 13: Knights of the Dinner Table

[*Advent of Code 2015 day 13*](https://adventofcode.com/2015/day/13) and [*solution megathread*](https://www.reddit.com/3wm0oy)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/master/2015/13/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/master?filepath=2015%2F13%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

...

In [6]:
testdata = """Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol.""".splitlines()

# inputdata = downloaded['input']

In [ ]:
# assert(my_part1_solution(testdata) == 330)

In [ ]:
# my_part1_solution(inputdata)

In [7]:
HTML(downloaded['part1_footer'])

## Part Two

In [ ]:
# HTML(downloaded['part2'])

In [ ]:
# HTML(downloaded['part2_footer'])

In [ ]:
# assert(my_part2_solution(testdata) == ...)

In [ ]:
# my_part2_solution(inputdata)